In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

We have two days 1 minute data for Friday (June 11, 2021) and Monday (June 14, 2021). NIFTY was in the range of 15600 - 15850, so we also have all 12 strikes in this range.

In [2]:
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
nifty = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]
ce600 = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY2161715600CE_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]
pe600 = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY2161715600PE_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]
ce650 = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY2161715650CE_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]
pe650 = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY2161715650PE_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]
ce700 = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY2161715700CE_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]
pe700 = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY2161715700PE_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]
ce750 = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY2161715750CE_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]
pe750 = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY2161715750PE_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]
ce800 = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY2161715800CE_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]
pe800 = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY2161715800PE_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]
ce850 = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY2161715850CE_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]
pe850 = pd.read_csv('../historical/straddles_every_15_minutes/NIFTY2161715850PE_minute.csv', parse_dates=['Date'], date_parser=dateparse).iloc[:751]

In [3]:
options = {'ce600' : 'ce600', 'pe600' : 'pe600', 
           'ce650' : 'ce650', 'pe650' : 'pe650',
           'ce700' : 'ce700', 'pe700' : 'pe700',
           'ce750' : 'ce750', 'pe750' : 'pe750',
           'ce800' : 'ce800', 'pe800' : 'pe800',
           'ce850' : 'ce850', 'pe850' : 'pe850'}

Confirming that NIFTY was within 15600-15850

In [4]:
# A function to round the NIFTY index value at a given point to the nearest 50
def roundMeUp(x):    
    base = 50
    return base * np.round(x.Open/base).astype(np.int64)

np.unique(np.array([roundMeUp(nifty.iloc[i]) for i in range(750)])) # 1 day has 375 minutes, 2 days = 750

array([15600, 15650, 15700, 15750, 15800, 15850], dtype=int64)

In [5]:
# We will take the strike based on the opening value of every 15th minute 
# of NIFTY index rounded to the nearest 50 value

def multipleStraddles(start, end, end2):    
    ce = []
    pe = []
    strikes = []
    strike_time = []
    end_ce = []
    end_pe = []
    for i in range(start, end, 15):
        curr_strike = roundMeUp(nifty.iloc[i])
        strikes.append(curr_strike)
        strike_time.append(nifty.iloc[i].TIME1)

        curr_ce = eval(options['ce' + str(curr_strike)[2:]])
        curr_pe = eval(options['pe' + str(curr_strike)[2:]])
        ce.append(curr_ce.iloc[i].Open)
        pe.append(curr_pe.iloc[i].Open)
        end_ce.append(curr_ce.iloc[end2].Open)
        end_pe.append(curr_pe.iloc[end2].Open)

    ce = np.array(ce)
    pe = np.array(pe)
    end_ce = np.array(end_ce)
    end_pe = np.array(end_pe)
    df = pd.DataFrame({'Time': strike_time,
                    'Strike': strikes,
                    'CALL': ce,
                    'PUT': pe,
                    'Initial': ce + pe,
                    'End': end_ce + end_pe,
                    'PnL': (ce + pe) - (end_ce + end_pe)})
    print('Total points covered:', round(df.PnL.sum()))
    return df

### Friday - range bound market

In [6]:
multipleStraddles(0, 375, 374)

Total points covered: 386


,Time,Strike,CALL,PUT,Initial,End,PnL
0,09:15:00,15800,90.10,124.00,214.10,178.30,35.80
1,09:30:00,15800,102.75,99.60,202.35,178.30,24.05
2,09:45:00,15800,113.00,91.20,204.20,178.30,25.90
3,10:00:00,15800,106.75,94.25,201.00,178.30,22.70
4,10:15:00,15800,104.90,94.20,199.10,178.30,20.80
5,10:30:00,15800,108.15,90.65,198.80,178.30,20.50
6,10:45:00,15800,106.30,91.55,197.85,178.30,19.55
7,11:00:00,15800,108.65,90.25,198.90,178.30,20.60
8,11:15:00,15800,106.10,90.95,197.05,178.30,18.75
9,11:30:00,15800,96.55,99.00,195.55,178.30,17.25


### Monday - a bit of trending day if first 20 minutes are ignored

In [7]:
multipleStraddles(375, 750, 750)

Total points covered: -245


,Time,Strike,CALL,PUT,Initial,End,PnL
0,09:15:00,15800,80.00,110.00,190.00,155.55,34.45
1,09:30:00,15650,103.50,90.40,193.90,222.00,-28.10
2,09:45:00,15700,79.00,105.45,184.45,194.00,-9.55
3,10:00:00,15700,92.85,86.10,178.95,194.00,-15.05
4,10:15:00,15750,73.55,99.15,172.70,168.90,3.80
5,10:30:00,15700,93.85,82.90,176.75,194.00,-17.25
6,10:45:00,15700,93.80,80.70,174.50,194.00,-19.50
7,11:00:00,15700,92.10,81.35,173.45,194.00,-20.55
8,11:15:00,15700,86.00,85.05,171.05,194.00,-22.95
9,11:30:00,15700,84.40,85.85,170.25,194.00,-23.75
